<a href="https://colab.research.google.com/github/Renukumar-R/database_query_with_nlp/blob/main/SQL_Database_Query_With_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Query Employee DB With NPL Using Langchain Framework

#### This Colab notebook demonstrates how to leverage the LangChain framework for querying an employee database using Natural Language Processing (NLP) techniques. The steps involve importing necessary Python packages, setting up embeddings, creating a vector database, and employing LangChain's tools for semantic similarity and few-shot learning.

Installing all the required Python packages for a project from a requirements.txt file and import

In [ ]:
pip install -r requirements.txt

In [2]:
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.embeddings import GooglePalmEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts.example_selector.semantic_similarity import SemanticSimilarityExampleSelector
from langchain.llms import GooglePalm
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from langchain.prompts import FewShotPromptTemplate
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.example_selector.base import BaseExampleSelector

In [30]:
google_api_key = 'YourAPIKey'

Bringing in a small set of examples and distinctive names from our database for filtering. Converting them into a vector database and retrieving similar examples from that database.

In [6]:
#Initializes GooglePalmEmbeddings with your API key.
embeddings = GooglePalmEmbeddings(google_api_key = google_api_key)

#Loads data from a CSV file using CSVLoader, extracting specified metadata columns.
loader = CSVLoader(file_path='Data_to_vector.csv',metadata_columns=['sex','company','employement_type','department'])
data = loader.load()

#Creates key vectors using FAISS based on the loaded data and GooglePalmEmbeddings.
keyvectors = FAISS.from_documents(documents=data,embedding=embeddings)

#Sets up a SemanticSimilarityExampleSelector for the key vectors.
key_selector = SemanticSimilarityExampleSelector(vectorstore=keyvectors,k=2)

In [7]:
#Load few shots from file and convert them into vectors using FAISS
with open('few_shots.txt', 'r') as file:
    few_shots = eval(file.read())
few_content = [" ".join(example.values()) for example in few_shots]
few_shots_vector = FAISS.from_texts(few_content, embeddings, metadatas=few_shots)

#Set up SemanticSimilarityExampleSelector for few shots
few_shots_selector = SemanticSimilarityExampleSelector(vectorstore=few_shots_vector,k=2)

In [8]:
#Initializes LLM with your API key.
llm = GooglePalm(google_api_key=google_api_key, temperature=0.2)

In [9]:
#Set up SQLDatabase connection
db = SQLDatabase.from_uri("sqlite:///employee_details.db")

In [10]:
#Checking the DB
db.run("SELECT name FROM personal_details LIMIT 3")

"[('Aabid',), ('Aadil',), ('Aadil Sefi',)]"

In [11]:
#Creating the prompt with samples to give comments of LLM
example_prompt = PromptTemplate(
    input_variables=["company", "department", "employement_type","sex","Question", "SQLQuery", "SQLResult","Answer"],
    template='You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.\nUnless the user specifies in the question a specific number of examples to obtain, query for all results as per MySQL. You can order the results to return the most informative data in the database.\nNever query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.\nPay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.\nPay attention to use CURDATE() function to get the current date, if the question involves "today".\n\nIf the column filter user input values closely align with the provided reference values, selectively incorporate only those mentioned values for filtering. Filter the columns strictly based on the user\'s specified criteria. Only consider the columns explicitly mentioned by the user for the filtering process. Adhere to the user\'s order of preference when applying the filters.\n\nCompany: {company}\nDepartment: {department}\nEmployement Type: {employement_type}\nSex: {sex}\n\nUse the following format:\n\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}\n\n')

In [12]:
example_prompt

PromptTemplate(input_variables=['Answer', 'Question', 'SQLQuery', 'SQLResult', 'company', 'department', 'employement_type', 'sex'], template='You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.\nUnless the user specifies in the question a specific number of examples to obtain, query for all results as per MySQL. You can order the results to return the most informative data in the database.\nNever query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.\nPay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.\nPay attention to use CURDATE() function to get the current date, if the question involv

In [13]:
#In Langchain multiple example selector doesn't support so create class to process multiple selector
class CombinedExampleSelector(BaseExampleSelector):
    def __init__(self, example_selectors):
        self.example_selectors = example_selectors

    def select_examples(self, input_variables):
        selected_examples = []
        for example_selector in self.example_selectors:
            examples = example_selector.select_examples(input_variables)
            selected_examples.append(examples)
        selected_examples = [{**dict1, **dict2} for dict1, dict2 in zip(selected_examples[0],selected_examples[1])]
        return selected_examples

    def add_example(self, example):
        for example_selector in self.example_selectors:
            example_selector.add_example(example)

combined_selector = CombinedExampleSelector([key_selector, few_shots_selector])

In [14]:
#Set up FewShotPromptTemplate with a combined example selector, example prompt, and specified input variables.
few_shot_prompt = FewShotPromptTemplate(
    example_selector= combined_selector,
    example_prompt=example_prompt,
    suffix=PROMPT_SUFFIX,
    input_variables=["input", "table_info"], #These variables are used in the prefix and suffix
)

In [15]:
#Initialise the chain model to query db
model = SQLDatabaseChain.from_llm(llm, db, verbose=True, prompt=few_shot_prompt)

Now ready to query the db with natural language

In [29]:
model.run('How many female employees have grater than 30 age in tesla and in AI department?')



> Entering new SQLDatabaseChain chain...
How many female employees have grater than 30 age in tesla and in AI department?
SQLQuery:SELECT count(*) FROM employement_details AS e JOIN personal_details AS p ON e.employee_id = p.employee_id WHERE e.company = 'Tesla, Inc.' AND e.department = 'AI' AND p.sex = 'Female' AND p.age > 30
SQLResult: [(47,)]
Answer:47
> Finished chain.


'47'